# Rainbow options with Direct Amplitude Loading
In this Notebook we will go through the implementation of the Direct Amplitude Loading Method for the rainbow option presented in [[1]](#QALROP)

### Introduction

In finance, a crucial aspect of asset pricing pertains to derivatives.
Derivatives are contracts whose value is contingent upon another source, known as the underlying.
The pricing of options, a specific derivative instrument, involves determining the fair market value (discounted payoff) of contracts affording their holders the right, though not the obligation, to buy (call) or sell (put) one or more underlying assets at a predefined strike price by a specified future expiration date (maturity date).
This process relies on mathematical models, considering variables like current asset prices, time to expiration, volatility, and interest rates.

## Data Definitions

The problem inputs are:
- `NUM_QUBITS`: the number of qubits representing an underlying asset
- `NUM_ASSETS`: the number of underlying assets
- `K`: the strike price
- `S0`: the arrays of underlying assets prices
- `dt`: the number of days to the maturity date
- `COV`: the covariance matrix that correlate the underlyings
- `MU_LOG_RET`: the array containing the mean of the log return of each underlyings

In [ ]:
import numpy as np
import scipy

NUM_QUBITS = 2
NUM_ASSETS = 2

K = 190
S0 = [193.97, 189.12]
dt = 250

COV = np.array([[0.000335, 0.000257], [0.000257, 0.000418]])
MU_LOG_RET = np.array([0.00050963, 0.00062552])

In [ ]:
MU = MU_LOG_RET * dt
CHOLESKY = np.linalg.cholesky(COV) * np.sqrt(dt)
SCALING_FACTOR = 1 / CHOLESKY[0, 0]

## Gaussian State preparation
Encode the probability distribution of a discrete multivariate random variable $W$ taking values in $\{w_0, .., w_{N-1}\}$ describing the assets' prices at the maturity date. The number of discretized values, denoted as $N$, depends on the precision of the state preparation module and is consequently connected to the number of qubits ($n=$) according to the formula $N=2^n$. 

$$
\sum_{i=0}^{N-1} \sqrt{p(w_i)}\left|w_i\right\rangle 
$$

In [ ]:
def gaussian_discretization(num_qubits, mu=0, sigma=1, stds_around_mean_to_include=3):
    lower = mu - stds_around_mean_to_include * sigma
    upper = mu + stds_around_mean_to_include * sigma
    num_of_bins = 2**num_qubits
    sample_points = np.linspace(lower, upper, num_of_bins + 1)

    def single_gaussian(x: np.ndarray, _mu: float, _sigma: float) -> np.ndarray:
        cdf = scipy.stats.norm.cdf(x, loc=_mu, scale=_sigma)
        return cdf[1:] - cdf[0:-1]

    non_normalized_pmf = (single_gaussian(sample_points, mu, sigma),)
    real_probs = non_normalized_pmf / np.sum(non_normalized_pmf)
    return sample_points[:-1], real_probs[0].tolist()


grid_points, probabilities = gaussian_discretization(NUM_QUBITS)

STEP_X = grid_points[1] - grid_points[0]
MIN_X = grid_points[0]

### SANITY CHECK

The process must be stopped if the strike price $K$ is greater than the maximum value reacheable by the assets during the simulation, to avoid meaningless results. The payoff is $0$ in this case, so there is no need to simulate.

In [ ]:
from IPython.display import Markdown

if K >= max(S0*np.exp(np.dot(CHOLESKY,[grid_points[-1]]*2) + MU)):
    display(Markdown('<font color=\'red\'> K always greater than the maximum asset values. Stop the run, the payoff is 0</font>'))

## Maximum Computation

### Precision utils

In [ ]:
FRAC_PLACES = 2

def round_factor(a):
    precision_factor = 2 ** FRAC_PLACES
    return round(a * precision_factor) / precision_factor


def floor_factor(a):
    precision_factor = 2 ** FRAC_PLACES
    return np.floor(a * precision_factor) / precision_factor

### Affine and maximum arithmetic definitions

Considering the time delta between the starting date ($t_0$) and the maturity date ($t$), we can express the return value $R_i$ for the $i$-th asset as $R_i = \mu_i + y_i$. Where:

  $\mu_i= (t-t_0)\tilde{\mu}_i$, being $\tilde{\mu}_i$ the expected daily log-return value. It can be estimated by considering the historical time series of log returns for the $i$-th asset.

$y_i$ is obtained through the dot product between the matrix $\mathbf{L}$ and the standard multivariate Gaussian sample:

$$    y_i = \Delta x \cdot \sum_kl_{ik}d_k + x_{min} \cdot \sum_k l_{ik} $$
$\Delta x$ is the Gaussian discretization step, $x_{min}$ is the lower Gaussian truncation value and $d_k \in [0,2^m-1]$ is the sample taken from the $k$-th standard Gaussian.
$l_{ik}$ is the $i,k$ entry of the matrix $\mathbf{L}$, defined as $\mathbf{L}=\mathbf{C}\sqrt{(t-t_0)}$, where $\mathbf{C}$ is the lower triangular matrix obtained by applying the Cholesky decomposition to the historical daily log-returns correlation matrix.

In [ ]:
from functools import reduce

from classiq import QNum, get_expression_numeric_attributes, qfunc, Output
from classiq.qmod.symbolic import max as qmax

a = STEP_X / SCALING_FACTOR
b = np.log(S0[0]) + MU[0] + MIN_X * CHOLESKY[0].sum()
c = (
        SCALING_FACTOR
        * (
        np.log(S0[1])
        + MU[1]
        - (np.log(S0[0]) + MU[0])
        + MIN_X * sum(CHOLESKY[1] - CHOLESKY[0])
    )
        / STEP_X
)
c = round_factor(c)


def get_affine_formula(assets, i):
    return reduce(
        lambda x, y: x + y,
        [
            assets[j] * round_factor(SCALING_FACTOR * CHOLESKY[i, j])
            for j in range(NUM_ASSETS)
            if CHOLESKY[i, j]
        ],
    )


def calculate_max_reg_type():
    x1 = QNum("x1", NUM_QUBITS, False, 0)
    x2 = QNum("x2", NUM_QUBITS, False, 0)
    expr = qmax(get_affine_formula([x1, x2], 0), get_affine_formula([x1, x2], 1) + c)
    size_in_bits, sign, fraction_digits = get_expression_numeric_attributes(
        [x1, x2], expr
    )
    return size_in_bits, fraction_digits


MAX_NUM_QUBITS = calculate_max_reg_type()[0]
MAX_FRAC_PLACES = calculate_max_reg_type()[1]

In [ ]:
@qfunc
def affine_max(x1: QNum, x2: QNum, res: Output[QNum]):
    res |= qmax(get_affine_formula([x1, x2], 0), get_affine_formula([x1, x2], 1) + c)

## Direct Method
The direct exponential amplitude loading encodes in $\tilde{f}$ the following function:
\begin{equation}
\tilde{f}(x)=
    \begin{cases}
        e^{-a\hat{x}}, & \text{if } \frac{x}{2^P} \geq \frac{\log(K) -b'}{b}\\
      Ke^{-(b'+ ax_{max})}, & \text{if } \frac{x}{2^P} < \frac{\log(K) -b'}{b}
    \end{cases}
\end{equation}

where $\hat{x}$ is the binary complement of $x$ ($\hat{x}=x-x_{max}$) and $x_{max}=2^R-1$, the maximum value that can be stored in $|x\rangle$ register. For loading $e^{-a\hat{x}}$ the $|r\rangle$ is initialized to all zeros and one controlled rotation for each qubit is performed as shown in Figure \ref{fig:circuit}. The rotations angles are: $\theta_i = 2\arccos \left({\sqrt{e^{-a2^i}}}\right)$. All the probabilities of getting a $|0\rangle^{\otimes{R}}$ in the $|r\rangle$ are then collected by a multi-controlled X (MCX) gate and stored in the $|1\rangle$ state of a target qubit.

In [ ]:
from classiq import X, RY, apply_to_all, CReal, QArray, repeat, QBit, Constraints, synthesize, create_model, allocate, inplace_prepare_state, within_apply, control, bind, show
from classiq.qmod.symbolic import acos, asin, sqrt, exp

@qfunc
def exponential_amplitude_loading(
    exp_rate: CReal, x: QArray[QBit], aux: QArray[QBit], res: QBit
) -> None:
    apply_to_all(X, x)
    repeat(
        x.len,
        lambda index: control(
            x[index],
            lambda: RY(2 * acos(1 / sqrt(exp(exp_rate * (2**index)))), aux[index])
        ),
    )
    apply_to_all(X, x)

    aux_num = QNum("aux_num", aux.len, False, 0)
    bind(aux, aux_num)
    control(aux_num == 0, lambda: X(res))
    bind(aux_num, aux)

In [ ]:
def get_payoff_expression(x, size, fraction_digits):
    payoff = sqrt(
        qmax(
            S0[0]
            * exp(
                STEP_X / SCALING_FACTOR * (2 ** (size - fraction_digits)) * x
                + (MU[0] + MIN_X * CHOLESKY[0].sum())
            ),
            K,
        )
    )
    return payoff


def get_strike_price_theta_direct(x: QNum):
    x_max = 1 - 1 / (2**x.size)
    payoff_max = get_payoff_expression(x_max, x.size, x.fraction_digits)

    return 2 * asin(np.sqrt(K) / payoff_max)


@qfunc
def direct_load_amplitudes(geq_reg: QNum, max_reg: QNum, aux_reg: QNum, ind_reg: QBit):
    exp_rate = (1 / (2**max_reg.fraction_digits)) * a
    control(
        geq_reg == 1,
        lambda: exponential_amplitude_loading(exp_rate, max_reg, aux_reg, ind_reg),
    )
    strike_price_theta = get_strike_price_theta_direct(max_reg)
    control(geq_reg == 0, lambda: RY(strike_price_theta, ind_reg))


@qfunc
def direct_payoff(max_reg: QNum, aux_reg: QNum, ind_reg: QBit):
    geq_reg = QBit("geq_reg")
    within_apply(
        lambda: asset_geq_strike_price(max_reg, geq_reg),
        lambda: direct_load_amplitudes(geq_reg, max_reg, aux_reg, ind_reg),
    )

@qfunc
def asset_geq_strike_price(
    x: QNum,
    res: Output[QBit],
) -> None:
    a = STEP_X / SCALING_FACTOR
    b = np.log(S0[0]) + MU[0] + MIN_X * CHOLESKY[0].sum()
    COMP_VALUE = (np.log(K) - b) / a
    res |= x > floor_factor(COMP_VALUE)

@qfunc
def rainbow_direct(
    x1: QNum,  
    x2: QNum,
    aux_reg: QNum,
    ind_reg: QBit,
) -> None:
    inplace_prepare_state(probabilities, 0, x1)
    inplace_prepare_state(probabilities, 0, x2)
    max_out = QNum("max_out")
    within_apply(
        lambda: affine_max(x1, x2, max_out),
        lambda: direct_payoff(max_out, aux_reg, ind_reg),
    )


@qfunc
def main(
    x1: Output[QNum],  
    x2: Output[QNum],
    aux_reg: Output[QNum],
    ind_reg: Output[QBit],
) -> None:
    allocate(MAX_NUM_QUBITS, aux_reg)
    allocate(NUM_QUBITS, x1)
    allocate(NUM_QUBITS, x2)
    allocate(1, ind_reg)
    rainbow_direct(x1, x2, aux_reg, ind_reg)

constraints = Constraints(max_width=23)
qmod = create_model(main, constraints=constraints)
print("Starting synthesis")
qprog = synthesize(qmod)
show(qprog)

## IQAE algorithm

In [ ]:
from classiq import cfunc, Z
from classiq.qmod.builtins.classical_execution_primitives import iqae, save


@qfunc
def qmci_oracle(ind: QBit):
    Z(ind)

@cfunc
def cmain():
    iqae_res = iqae(epsilon=0.05, alpha=0.1)
    save({"iqae_res": iqae_res})

In [ ]:
from classiq import CInt, QCallable, grover_operator, power

@qfunc
def grover_algorithm(
    k: CInt,
    oracle_operand: QCallable[QArray[QBit]],
    sp_operand: QCallable[QArray[QBit]],
    x: QArray[QBit],
):
    sp_operand(x)
    power(k, lambda: grover_operator(oracle_operand, sp_operand, x))

In [ ]:
def get_main():
    @qfunc
    def main(
        k: CInt,
        ind_reg: Output[QBit],
    ) -> None:
        full_reg = QArray("full_reg")
        allocate(2 * NUM_QUBITS + MAX_NUM_QUBITS + 1, full_reg)
        grover_algorithm(
            k,
            lambda x: qmci_oracle(x[x.len - 1]),
            lambda x: rainbow_direct(
                x[0:NUM_QUBITS],
                x[NUM_QUBITS : 2 * NUM_QUBITS],
                x[2 * NUM_QUBITS : 2 * NUM_QUBITS + MAX_NUM_QUBITS],
                x[x.len - 1],
            ),
            full_reg,
        )
        state_reg = QArray("state_reg")
        bind(full_reg, [state_reg, ind_reg])

    return main

In [ ]:
from classiq import execute

def synthesize_and_execute(post_process):
    constraints = Constraints(max_width=25)
    qmod = create_model(
        get_main(),
        constraints=constraints,
        classical_execution_function=cmain,
    )
    print("Starting synthesis")
    qprog = synthesize(qmod)
    show(qprog)
    print("Starting execution")
    res = execute(qprog).result()
    iqae_res = res[0].value
    parsed_res = post_process(res[0].value)

    return (qmod, qprog, iqae_res, parsed_res)

## Post Process
We need to add to the post-processing function a term:
\begin{equation}
\begin{split}
&\mathbb{E} \left[\max\left(e^{b \cdot z}, Ke^{-b'}\right) \right] e^{b'} - K \\
= &\mathbb{E} \left[\max\left(e^{-a\hat{x}}, Ke^{-b'-ax_{max}}\right) \right]e^{b'+ ax_{max}} - K 
\end{split}
\end{equation}

In [ ]:
import sympy

def parse_result_direct(iqae_res):
    payoff_expression = f"sqrt(max({S0[0]} * exp({STEP_X / SCALING_FACTOR * (2 ** (MAX_NUM_QUBITS - MAX_FRAC_PLACES))} * x + ({MU[0]+MIN_X*CHOLESKY[0].sum()})), {K}))"
    payoff_func = sympy.lambdify(sympy.symbols("x"), payoff_expression)
    payoff_max = payoff_func(1 - 1 / (2**MAX_NUM_QUBITS)) 

    option_value = iqae_res.estimation * (payoff_max**2) - K
    confidence_interval = np.array(iqae_res.confidence_interval) * (payoff_max**2) - K
    return (option_value, confidence_interval)

# Run Method

In [ ]:
qmod_direct, qprog_direct, iqae_res_direct, parsed_res_direct = synthesize_and_execute(
    parse_result_direct
)
result, conf_interval = parsed_res_direct
print(f"raw iqae results: {iqae_res_direct.estimation} with confidence interval {iqae_res_direct.confidence_interval}")
print(f"option estimated value: {result} with confidence interval {conf_interval}")

## References

<a name='QALROP'>[1]</a>: [Francesca Cibrario et al., Quantum Amplitude Loading for Rainbow Options Pricing. Preprint](https://arxiv.org/abs/2402.05574v2)
